*   **Año:** [INSERTAR AÑO]
*   **Alumno/a:** [INSERTAR NOMBRE]
*   **Legajo:** [LEGAJO]

# Pydantic
Pydantic es una libreria rapida y extensible que nos permite validar datos usando tipos de datos de Python.


Primero, importe `pydantic`.

In [ ]:
#Escribir el codigo para responder la pregunta en esta celda
import pydantic

Supongamos que tenemos una lista de clientes (llamemoslo en el codigo `Client`). Los clientes tienen dos campos: DNI (un entero) y nacionalidad (un `string`). Cree el modelo base de `Client` (en forma de clases de Python).

In [ ]:
#Escribir el codigo para responder la pregunta en esta celda
from pydantic import BaseModel
class Cliente(BaseModel):
    dni: int
    nacionalidad: str

Cree a un usuario con documento 39.755.010 y nacionalidad 'Argentina'. Muestre todos sus campos.

In [ ]:
#Escribir el codigo para responder la pregunta en esta celda
cliente = Cliente(dni=39755010, nacionalidad="Argentina")
print(cliente)

dni=39755010 nacionalidad='Argentina'


Intente crear al usuario con un documento en forma de `string`. Deberia fallar...

In [ ]:
#Escribir el codigo para responder la pregunta en esta celda
from pydantic import BaseModel, StrictInt

class Cliente(BaseModel):
    dni: StrictInt
    nacionalidad: str

cliente1 = Cliente(dni="45327309", nacionalidad="Paraguay")
#Hay que cambiar el import y hacer que dni sea estricto en Str

ValidationError: 1 validation error for Cliente
dni
  Input should be a valid integer [type=int_type, input_value='45327309', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_type

Hemos detectado que ciertos clientes tienen nacionalidades que no existen. Ademas, hay numeros de documento negativos y se tiene que poder agregar la fecha de registro de los clientes (que no pueden ser del futuro). Cambiar la definicion del cliente para que estas cosas no ocurran. Despues de la siguiente celda, cree otras 3 mas probando un caso donde deberia funcionar y otros dos en los que no.

In [ ]:
from datetime import date

class Cliente:
    NACIONALIDADES_VALIDAS = ["Argentina", "Chile", "Brasil", "Uruguay"]

    def __init__(self, dni: int, nacionalidad: str, fecha_registro: date):
        if dni < 0:
            raise ValueError("El DNI debe ser un número positivo.")
        self.dni = dni

        if nacionalidad not in Cliente.NACIONALIDADES_VALIDAS:
            raise ValueError(f"Nacionalidad '{nacionalidad}' no es válida. Debe ser una de {Cliente.NACIONALIDADES_VALIDAS}.")
        self.nacionalidad = nacionalidad

        if fecha_registro > date.today():
            raise ValueError("La fecha de registro no puede ser una fecha futura.")
        self.fecha_registro = fecha_registro

    def __repr__(self):
        return f"Client(DNI={self.dni}, Nacionalidad='{self.nacionalidad}', Fecha de Registro={self.fecha_registro})"

In [ ]:
#Escribir el codigo para responder la pregunta en esta celda
#VALIDO
from datetime import date

cliente1 = Cliente(39755020, "Argentina", date(2023, 11, 14))
print(cliente1)

Client(DNI=39755020, Nacionalidad='Argentina', Fecha de Registro=2023-11-14)


In [ ]:
#Escribir el codigo para responder la pregunta en esta celda
#INVALIDO
from datetime import date

cliente2 = Cliente(-39755010, "Argentina", date(2023, 11, 14))
print(cliente2)

ValueError: El DNI debe ser un número positivo.

In [ ]:
#Escribir el codigo para responder la pregunta en esta celda
#INVALIDO
from datetime import date

cliente3 = Cliente(39755010, "Peru", date(2023, 11, 14))
print(cliente3)

ValueError: Nacionalidad 'Peru' no es válida. Debe ser una de ['Argentina', 'Chile', 'Brasil', 'Uruguay'].

# Pandera
Hacer lo que se pidio en la ultima celda anterior, pero con Pandera.

In [ ]:
#Escribir el codigo para responder la pregunta en esta celda

!pip install --upgrade pandera

from datetime import date
import pandera as pa
import pandas as pd

# Clase Client sin validaciones internas
class Client:
    def _init_(self, dni: int, nacionalidad: str, fecha_registro: date):
        self.dni = dni
        self.nacionalidad = nacionalidad
        self.fecha_registro = fecha_registro

    def _repr_(self):
        return f"Client(DNI={self.dni}, Nacionalidad='{self.nacionalidad}', Fecha de Registro={self.fecha_registro})"


# Definición del esquema de validación con pa.DataFrameSchema
client_schema = pa.DataFrameSchema({
    "dni": pa.Column(int, checks=pa.Check.ge(0)),  # DNI positivo
    "nacionalidad": pa.Column(str, checks=pa.Check.isin(["Argentina", "Chile", "Brasil", "Uruguay", "Paraguay"])),
    "fecha_registro": pa.Column(date, checks=pa.Check.le(date.today()))  # Fecha no futura
})


# Función para validar un cliente usando Pandera
def validate_client(client):
    # Crear un DataFrame de una sola fila para el cliente
    df_client = pd.DataFrame([{
        "dni": client.dni,
        "nacionalidad": client.nacionalidad,
        "fecha_registro": client.fecha_registro
    }])

    # Validar el DataFrame contra el esquema
    try:
        client_schema.validate(df_client)
        print("Cliente válido:", client)
    except pa.errors.SchemaError as e:
        print("Error de validación:", e)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 4.7 MB/s eta 0:00:00


In [ ]:
#INVALIDO
from datetime import date

cliente3 = Cliente(39755010, "Peru", date(2023, 11, 14))
print(cliente3)


ValueError: Nacionalidad 'Peru' no es válida. Debe ser una de ['Argentina', 'Chile', 'Brasil', 'Uruguay'].

In [ ]:
#Escribir el codigo para responder la pregunta en esta celda
#VALIDO
from datetime import date

cliente1 = Cliente(39755020, "Argentina", date(2023, 11, 14))
print(cliente1)

Client(DNI=39755020, Nacionalidad='Argentina', Fecha de Registro=2023-11-14)
